In [1]:
from dask.distributed import fire_and_forget
from dask.distributed import Client, Variable , Queue , Lock ,LocalCluster
from dask_jobqueue import SLURMCluster
from dask.distributed import  utils_perf
from dask.distributed import Client, LocalCluster
import dask
import redis
from bloom_filter2 import BloomFilter
import lzma
from dask import dataframe as dd

In [2]:
#!/usr/bin/env python
# coding: utf-8
import sys
sys.path.append('../../..')
#sys.path.append( '/home/cactuskid13/miniconda3/pkgs/')
print(sys.path)
import ete3
import random
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import itertools
import dask
import warnings
from dask import dataframe as dd
import pickle
path = '/work/FAC/FBM/DBC/cdessim2/default/dmoi/'

['/work/FAC/FBM/DBC/cdessim2/default/dmoi/projects/RDF_Graphnet/rdf2network', '/work/FAC/FBM/DBC/cdessim2/default/dmoi/miniconda3/envs/ML2/lib/python310.zip', '/work/FAC/FBM/DBC/cdessim2/default/dmoi/miniconda3/envs/ML2/lib/python3.10', '/work/FAC/FBM/DBC/cdessim2/default/dmoi/miniconda3/envs/ML2/lib/python3.10/lib-dynload', '', '/work/FAC/FBM/DBC/cdessim2/default/dmoi/miniconda3/envs/ML2/lib/python3.10/site-packages', '../../..']


In [3]:
from dask.distributed import fire_and_forget
from dask.distributed import Client, Variable , Queue , Lock ,LocalCluster
from dask_jobqueue import SLURMCluster
from dask.distributed import  utils_perf
from dask.distributed import Client, LocalCluster
import dask
import redis
from bloom_filter2 import BloomFilter
import lzma
from dask import dataframe as dd

#using distributed computation on a slurm cluster here. This is my particular config. You will need to alter this: https://distributed.dask.org/en/stable/
NCORE = 4
print('deploying cluster')
cluster = SLURMCluster(
    #change theses settings for your cluster
    walltime='4:00:00',
    n_workers = NCORE,
    cores=NCORE,
    processes = NCORE,
    interface='ib0',
    memory="300GB",
    #job_script_prologue=[
    #' source /work/FAC/FBM/DBC/cdessim2/default/dmoi/miniconda3/etc/profile.d/conda.sh  ' ,
    #'conda activate ML2'
    #],
    #scheduler_options={'interface': 'ens2f0' },
    #if gpu node
    scheduler_options={'interface': 'ens3f0np0' },
    #extra=[ "--lifetime-stagger", "4m"]
)
print(cluster.job_script())
print(cluster)
cluster.scale(jobs = 100)
print(cluster.dashboard_link)
client = Client(cluster , timeout='450s' , set_as_default=True )

deploying cluster
#!/usr/bin/env bash

#SBATCH -J dask-worker
#SBATCH -n 1
#SBATCH --cpus-per-task=4
#SBATCH --mem=47G
#SBATCH -t 4:00:00

/work/FAC/FBM/DBC/cdessim2/default/dmoi/miniconda3/envs/ML2/bin/python -m distributed.cli.dask_worker tcp://10.203.101.82:35359 --nthreads 1 --nworkers 4 --memory-limit 11.64GiB --name dummy-name --nanny --death-timeout 60 --interface ib0

SLURMCluster(1cf67d4c, 'tcp://10.203.101.82:35359', workers=0, threads=0, memory=0 B)
http://10.203.101.82:8787/status


In [4]:
#find which species each of the cogs has an interaction in
link_df = dd.read_csv(path + 'datasets/STRING/protein.links.full.v11.5.txt',  blocksize=75e6 , header = 0, sep = ' ')

print(link_df)

Dask DataFrame Structure:
                  protein1 protein2 neighborhood neighborhood_transferred fusion cooccurence homology coexpression coexpression_transferred experiments experiments_transferred database database_transferred textmining textmining_transferred combined_score
npartitions=20540                                                                                                                                                                                                                                             
                    object   object        int64                    int64  int64       int64    int64        int64                    int64       int64                   int64    int64                int64      int64                  int64          int64
                       ...      ...          ...                      ...    ...         ...      ...          ...                      ...         ...                     ...      ...                  ...    

In [ ]:
#compute bloom filters for protein pairs
@dask.delayed
def protlinks_species( df ):
    df = df[~df['protein1'].isna()]
    df = df[~df['protein2'].isna()]
    df.protein1 = df.protein1.map(lambda x:str(x))
    df.protein2 = df.protein2.map(lambda x:str(x))
    df['protlinks'] = df.protein1 + '_' + df.protein2 
    ret = set(df.protlinks.unique())
    return ret

@dask.delayed
def return_filter(protlinks, verbose = True):
    if type( protlinks ) == tuple:
        protlinks = protlinks[0]
    b=BloomFilter(max_elements=10**8, error_rate=0.001 ,start_fresh = True)
    for p in protlinks:
        b.add( p )
    retlen = len(protlinks)
    return   b , retlen

@dask.delayed
def sumfilter(f1,f2, total ):
    if type( f1 ) == tuple:
        f1 = f1[0]
    if type( f2 ) == tuple:
        f2 = f2[0]
    f3 = f1.__ior__(f2)
    return f3 , total

def treesum(totalfilter):
    print(len(totalfilter))
    while len(totalfilter)>1:
        next_round= []
        for i in range(0,len(totalfilter),2):
            if i+1 < len(totalfilter):
                next_round.append( sumfilter( totalfilter[i][0] , totalfilter[i+1][0] , totalfilter[i][1]+totalfilter[i+1][1]  ) )
        if len(totalfilter) % 2 !=0:
            next_round.append(totalfilter[-1])
        totalfilter = next_round
        print(len(totalfilter))
    return totalfilter

b=BloomFilter(max_elements=10**8, error_rate=0.001 ,start_fresh = True)
partitions  = link_df.to_delayed()
print('map cogs')
res1 = [ protlinks_species(p) for p in partitions ]
print('done')
print('make filters')
res2 = [ return_filter(p) for p in res1 ]


finals =[]

for chunk in range(int(len(res2)/1024)+1):
    print(chunk*1024)
    res3 = res2[chunk*1024:(chunk+1)*1024]
    res4 = treesum(res3)
    res4 = dask.compute(res4)
    print(res4)
    finals.append(res4[0])
    
dask.compute(*finals)

with open('bloomfinal_big.pkl' , 'wb' ) as finalout:
    finalout.write(pickle.dumps(finals))

map cogs
done
make filters
0
1024
512
256
128
64
32
16
8
4
2
1
([(BloomFilter(ideal_num_elements_n=100000000, error_rate_p=0.001000, num_bits_m=1437758757), 1089746228)],)
1024
1024
512
256
128
64
32
16
8
4
2
1
([(BloomFilter(ideal_num_elements_n=100000000, error_rate_p=0.001000, num_bits_m=1437758757), 986466698)],)
2048
1024
512
256
128
64
32
16
8
4
2
1
([(BloomFilter(ideal_num_elements_n=100000000, error_rate_p=0.001000, num_bits_m=1437758757), 1012062058)],)
3072
1024
512
256
128
64
32
16
8
4
2
1
([(BloomFilter(ideal_num_elements_n=100000000, error_rate_p=0.001000, num_bits_m=1437758757), 993933079)],)
4096
1024
512
256
128
64
32
16
8
4
2
1
([(BloomFilter(ideal_num_elements_n=100000000, error_rate_p=0.001000, num_bits_m=1437758757), 1027008149)],)
5120
1024
512
256
128
64
32
16
8
4
2
1
([(BloomFilter(ideal_num_elements_n=100000000, error_rate_p=0.001000, num_bits_m=1437758757), 1022627788)],)
6144
1024
512
256
128
64
32
16
8
4
2
1
([(BloomFilter(ideal_num_elements_n=100000000, erro